[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PetiteIA/schema_mechanism/blob/master/experiments/small_loop.ipynb)

# THE SMALL LOOP ENVIRONMENT

In [58]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from ipywidgets import Button, HBox,VBox, Output
from IPython.display import display

## Define the actions

In [59]:
FORWARD = 0
TURN_LEFT = 1
TURN_RIGHT = 2
FEEL_FRONT = 3
FEEL_LEFT = 4
FEEL_RIGHT = 5

## Define the environment 

In [115]:
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3
FEELING = 2
BUMPING = 3

class SmallLoop():
    def __init__(self, poX, poY, direction):
        self.grid = np.array([
            [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
            [1, 0, 0, 0, 1, 1, 0, 0, 1, 1],
            [1, 0, 1, 0, 0, 1, 1, 0, 0, 1],
            [1, 0, 1, 1, 0, 1, 1, 1, 0, 1],
            [1, 0, 0, 0, 0, 1, 1, 1, 0, 1],
            [1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
            [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
            [1, 0, 0, 1, 1, 1, 1, 1, 0, 1],
            [1, 1, 0, 0, 0, 0, 0, 0, 0, 1],
            [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
        ])
        self.maze = self.grid.copy()
        self.poX = poX
        self.poY = poY
        self.direction = direction
        self.cmap = ListedColormap(['white', 'green', 'yellow', 'red'])
        self.norm = BoundaryNorm([-0.5, 0.5, 1.5, 2.5, 3.5], self.cmap.N)

    def step(self, action):
        # print('before:', self.agent_position.strPosition(), action_dcit[action])
        self.maze[:,:] = self.grid
        result = 0
        
        if action == FORWARD:  # move forward
            # print('the action is move forward')
            # print(str(self.position.pointX)+': '+ str(self.position.pointY)+ ' ' +self.direction, action)
        
            if self.direction == LEFT:
                if self.maze[self.poX][self.poY - 1] == 0:
                    self.poY -= 1
                else:
                    self.maze[self.poX][self.poY - 1] = BUMPING
                    result = 1
            elif self.direction == DOWN:
                if self.maze[self.poX + 1][self.poY] == 0:
                    self.poX += 1
                else:
                    self.maze[self.poX + 1][self.poY] = BUMPING
                    result = 1
            elif self.direction == RIGHT:
                if self.maze[self.poX][self.poY + 1] == 0:
                    self.poY += 1
                else:
                    self.maze[self.poX][self.poY + 1] = BUMPING
                    result = 1
            elif self.direction == UP:
                if self.maze[self.poX - 1][self.poY] == 0:
                    self.poX -= 1
                else:
                    self.maze[self.poX - 1][self.poY] = BUMPING
                    result = 1
            # print(str(self.position.pointX)+': '+ str(self.position.pointY)+ ' ' +self.direction, action)
        elif action == TURN_RIGHT:
            if self.direction == LEFT:
                self.direction = UP
            elif self.direction == DOWN:
                self.direction = LEFT
            elif self.direction == RIGHT:
                self.direction = DOWN
            elif self.direction == UP:
                self.direction = RIGHT
        elif action == TURN_LEFT:
            if self.direction == LEFT:
                self.direction = DOWN
            elif self.direction == DOWN:
                self.direction = RIGHT
            elif self.direction == RIGHT:
                self.direction = UP
            elif self.direction == UP:
                self.direction = LEFT
        elif action == FEEL_FRONT:
            if self.direction == LEFT:
                if self.maze[self.poX][self.poY - 1] != 0:
                    result = 1
                self.maze[self.poX][self.poY - 1] = FEELING
            elif self.direction == DOWN:
                if self.maze[self.poX + 1][self.poY] != 0:
                    result = 1
                self.maze[self.poX + 1][self.poY] = FEELING
            elif self.direction == RIGHT:
                if self.maze[self.poX][self.poY + 1] != 0:
                    result = 1
                self.maze[self.poX][self.poY + 1] = FEELING
            elif self.direction == UP:
                if self.maze[self.poX - 1][self.poY] != 0:
                    result = 1
                self.maze[self.poX - 1][self.poY] = FEELING
        elif action == FEEL_LEFT:
            if self.direction == LEFT:
                if self.maze[self.poX + 1][self.poY] != 0:
                    result = 1
                self.maze[self.poX + 1][self.poY] = FEELING
            elif self.direction == DOWN:
                if self.maze[self.poX][self.poY + 1] != 0:
                    result = 1
                self.maze[self.poX][self.poY + 1] = FEELING
            elif self.direction == RIGHT:
                if self.maze[self.poX - 1][self.poY] != 0:
                    result = 1
                self.maze[self.poX - 1][self.poY] = FEELING
            elif self.direction == UP:
                if self.maze[self.poX][self.poY - 1] != 0:
                    result = 1
                self.maze[self.poX][self.poY - 1] = FEELING
        elif action == FEEL_RIGHT:
            if self.direction == LEFT:
                if self.maze[self.poX - 1][self.poY] != 0:
                    result = 1
                self.maze[self.poX - 1][self.poY] = FEELING
            elif self.direction == DOWN:
                if self.maze[self.poX][self.poY - 1] != 0:
                    result = 1
                self.maze[self.poX][self.poY - 1] = FEELING
            elif self.direction == RIGHT:
                if self.maze[self.poX + 1][self.poY] != 0:
                    result = 1
                self.maze[self.poX + 1][self.poY] = FEELING
            elif self.direction == UP:
                if self.maze[self.poX][self.poY + 1] != 0:
                    result = 1
                self.maze[self.poX][self.poY + 1] = FEELING
        print(f"Line: {self.poX}, Column: {self.poY}, direction: {self.direction}")
        # return self.position,

        return result  
    
    def display(self):
        out.clear_output(wait=True)
        with out:
            fig, ax = plt.subplots()
            # ax.set_xticks([])
            # ax.set_yticks([])
            # ax.axis('off')
            # ax.imshow(self.maze, cmap='Greens', vmin=0, vmax=2)
            ax.imshow(self.maze, cmap=self.cmap, norm=self.norm)
            if self.direction == LEFT:
                # Y is column and X is line
                plt.scatter(self.poY, self.poX, s=200, marker='<')
            elif self.direction == DOWN:
                plt.scatter(self.poY, self.poX, s=200, marker='v')
            elif self.direction == RIGHT:
                plt.scatter(self.poY, self.poX, s=200, marker='>')
            else: # UP
                plt.scatter(self.poY, self.poX, s=200, marker='^')
            plt.show()


## Initialize and display the environment

In [116]:
small_loop = SmallLoop(1, 1, 0)

out = Output()
small_loop.display()
display(out)

Output()

## Enact actions

Use `Ctrl+Enter` to run the cell below and stay on it.

In [114]:
result = small_loop.step(FORWARD) # 0
small_loop.display()
print(f"outcome: {result}") 

Line: 1, Column: 1, direction: 3
outcome: 1


In [109]:
result = small_loop.step(TURN_LEFT) # 1
small_loop.display()
print(f"outcome: {result}") 

Line: 1, Column: 1, direction: 1
outcome: 0


In [117]:
result = small_loop.step(TURN_RIGHT) # 2
small_loop.display()
print(f"outcome: {result}") 

Line: 1, Column: 1, direction: 3
outcome: 0


In [108]:
result = small_loop.step(FEEL_FRONT) # 3
small_loop.display()
print(f"outcome: {result}") 

Line: 1, Column: 1, direction: 0
outcome: 1


In [105]:
result = small_loop.step(FEEL_LEFT) # 4
small_loop.display()
print(f"outcome: {result}") 

Line: 1, Column: 1, direction: 0
outcome: 0


In [107]:
result = small_loop.step(FEEL_RIGHT) # 5
small_loop.display()
print(f"outcome: {result}") 

Line: 1, Column: 1, direction: 0
outcome: 1


# Test (ne marche pas)

In [118]:
def forward():
    small_loop.step(FORWARD)
    small_loop.display()

def turn_left():
    small_loop.step(TURN_LEFT)
    small_loop.display()

def turn_right():
    small_loop.step(TURN_RIGHT)
    small_loop.display()

# Create Start and Stop buttons
forward_button = Button(description="Forward")
left_button = Button(description="Turn left")
right_button = Button(description="Turn right")

# Bind button clicks to functions
forward_button.on_click(forward)
left_button.on_click(turn_left)
right_button.on_click(turn_right)

controls = HBox([forward_button, left_button, right_button])
display(controls)